# Finetune VAE models

## importing the libraries

In [2]:
import os
ml_code_path='/home/leilapirhaji/mz_embed_engine/ml'
os.chdir(ml_code_path)

import pandas as pd
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from get_pretrain_encoder import get_pretrain_input_data
from models_VAE import VAE

%who

[neptune] [warning] NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.


DataLoader	 LabelEncoder	 TensorDataset	 VAE	 finetune_save_dir	 get_pretrain_input_data	 input_data_location	 label_encoder	 ml_code_path	 
optim	 os	 pd	 torch	 


## getting Finetune datasets

In [8]:

input_data_location='/home/leilapirhaji/PROCESSED_DATA_2'
pretrain_save_dir='/home/leilapirhaji/pretrained_VAE_models' 


(X_data_all, y_data_all, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)=get_pretrain_input_data(input_data_location)


X_data_train.shape, y_data_train.shape, X_data_val.shape, y_data_val.shape, X_data_test.shape, y_data_test.shape


/home/leilapirhaji/mz_embed_engine/ml/get_pretrain_encoder.py:60: DtypeWarning: Columns (1,8,28,30,31,32,33,34,46,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  y_data_all = pd.read_csv(pretrain_y_all, index_col=0)


((14391, 2736),
 (14391, 58),
 (3074, 2736),
 (3074, 58),
 (3083, 2736),
 (3083, 58))

## pre-train VAE models

In [9]:
X_data_train.shape, y_data_train.shape, X_data_val.shape, y_data_val.shape, X_data_test.shape, y_data_test.shape

((14391, 2736),
 (14391, 58),
 (3074, 2736),
 (3074, 58),
 (3083, 2736),
 (3083, 58))

In [ ]:
import importlib
import train_pretrain_vae
importlib.reload(train_pretrain_vae)
from train_pretrain_vae import pretrain_vae


# Simulated data for example purposes

pretrain_vae(X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test, 
                latent_size=256, 
                num_hidden_layers=3, dropout_rate=0.2, activation='leakyrelu', 
                use_batch_norm=True, epochs=3, learning_rate=1e-3, 
                save_path=pretrain_save_dir, model_name='vae_model.pth')
